#Installs + imports

In [1]:
pip install gspread oauth2client

In [2]:
!pip install load_dotenv

In [3]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [4]:
!pip install requests

In [5]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c6cd6bdefbd4059e03299c34776a4c5209665c2dc14e326c5440f775ae0df567
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [50]:
import requests
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
model = SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']


creds = ServiceAccountCredentials.from_json_keyfile_name('/content/matchmaking-405000-bf0da54925d2.json', scope)
client = gspread.authorize(creds)


sheet = client.open("Matchmaking").get_worksheet(0)


community = sheet.get_all_records()


print(community)

# data = pd.read_csv('/content/Matchmaking.csv', ';')


[{'Name': 'Emma Johnson', 'ExpertiseTweet': 'Exploring the digital frontier with #DigitalMarketing, #BrandDevelopment, and #UXDesign. Bringing brands to life online!', 'Question1': 'How do you measure and track customer engagement effectively in the early stages of your startup?', 'Question2': 'What strategies have you found most effective for building and maintaining a strong brand identity?', 'Question3': 'Can you share any user experience pitfalls to avoid when designing an app for a non-technical audience?'}, {'Name': 'Arjun Patel', 'ExpertiseTweet': 'Diving deep into #FinTech, #Blockchain, and #RegulatoryCompliance. Innovating for the financial future!', 'Question1': 'What are the key challenges you faced when seeking funding for a FinTech startup, and how did you overcome them?', 'Question2': 'How do you stay ahead of the curve with blockchain technology trends and their applications?', 'Question3': 'Can you discuss some strategies to navigate the complexities of regulatory compl

# Matching

In [43]:
df = pd.DataFrame(community)

In [44]:
model = SentenceTransformer('all-MiniLM-L6-v2')

expertise_embeddings = model.encode(df['ExpertiseTweet'].tolist())

question_texts = []
for _, row in df.iterrows():
    question_texts.extend(row[['Question1', 'Question2', 'Question3']].fillna('').tolist())

question_embeddings = model.encode(question_texts)

similarity_matrix = cosine_similarity(question_embeddings, expertise_embeddings)

# Assign -1 to the similarities where expertise and questions belong to the same person


In [45]:
num_people = len(df)
for i in range(num_people):
    for j in range(3):
        similarity_matrix[i*3+j, i] = -1  # Set -1 for self comparisons

In [46]:
# Transpose the matrix for readability
similarity_matrix = similarity_matrix.T

# Create a DataFrame to hold the similarity scores
rows = [f'{name}' for name in df['Name']]
# Generate columns names in the format Q1_Name, Q2_Name, Q3_Name for each person
columns = [f'Q{j+1}_{name}' for name in df['Name'] for j in range(3)]
similarity_df = pd.DataFrame(similarity_matrix, index=rows, columns=columns)

similarity_df.to_csv('similarity_matrix.csv')

In [47]:
similarity_df

,Q1_Emma Johnson,Q2_Emma Johnson,Q3_Emma Johnson,Q1_Arjun Patel,Q2_Arjun Patel,Q3_Arjun Patel,Q1_Sofia Ramirez,Q2_Sofia Ramirez,Q3_Sofia Ramirez,Q1_James Lee,...,Q3_Benjamin Garcia,Q1_Walid,Q2_Walid,Q3_Walid,Q1_Loick avec un K,Q2_Loick avec un K,Q3_Loick avec un K,Q1_Lola,Q2_Lola,Q3_Lola
Emma Johnson,-1.000000,-1.000000,-1.000000,0.293042,0.240744,0.159477,0.251012,0.115331,0.273958,0.228582,...,0.158736,0.177991,0.480561,0.028379,0.304946,0.045773,0.203697,0.160453,0.416214,0.189147
Arjun Patel,0.092561,0.242886,0.150931,-1.000000,-1.000000,-1.000000,0.326956,0.232775,0.361852,0.243198,...,0.290449,0.097045,0.142388,0.023606,0.131860,0.006949,0.202236,0.143324,0.225807,0.074661
Sofia Ramirez,0.148660,0.268901,0.108495,0.272418,0.365126,0.322030,-1.000000,-1.000000,-1.000000,0.213051,...,0.398001,0.150250,0.056593,0.045804,0.144081,0.068596,0.166528,0.247096,0.266843,-0.026219
James Lee,0.153549,0.116538,0.123491,0.180891,0.177446,0.210683,0.148880,-0.005686,0.198532,-1.000000,...,0.121379,-0.036931,0.179696,-0.032326,0.273107,0.143796,0.158726,0.070190,0.125511,0.113963
Lucas Hernandez,0.251674,0.564108,0.143826,0.217415,0.103354,0.159444,0.245006,0.101304,0.277853,0.152484,...,0.170747,0.275322,0.247284,0.104874,0.251257,0.061708,0.188442,0.352437,0.383741,0.094629
Mia Gonzalez,0.133350,0.246806,0.253539,0.052013,0.106292,0.042311,0.126685,0.009079,0.107168,0.108301,...,0.080782,0.296381,0.464950,0.357802,0.229220,0.014469,0.155245,0.294643,0.182001,0.184115
Liam Smith,0.285627,0.444259,0.183690,0.203178,0.234835,0.195329,0.317285,0.154137,0.246991,0.229758,...,0.239116,0.171625,0.209423,0.025237,0.246330,0.081160,0.211822,0.154323,0.385346,0.067124
Olivia Brown,0.234351,0.288534,0.403215,0.316549,0.293059,0.136419,0.263221,0.105125,0.275858,0.287014,...,0.168503,0.209057,0.208450,0.057969,0.250708,0.111589,0.193805,0.206873,0.394356,0.065627
Noah Wilson,0.098013,0.184678,0.079551,0.250240,0.277227,0.211449,0.268947,0.102582,0.243822,0.227428,...,0.149431,0.035174,0.262831,0.002173,0.197806,0.072087,0.196069,0.134265,0.190467,0.103263
Ava Martinez,0.361750,0.358758,0.146871,0.307406,0.266055,0.195532,0.305135,0.186732,0.264991,0.218642,...,0.246780,0.169543,0.184650,0.072970,0.200975,0.039369,0.238271,0.166341,0.427270,0.183103


In [48]:
similarity_df.iloc[18]

Q1_Emma Johnson        0.178009
Q2_Emma Johnson        0.227996
Q3_Emma Johnson        0.377680
Q1_Arjun Patel         0.138592
Q2_Arjun Patel         0.209972
Q3_Arjun Patel         0.008938
Q1_Sofia Ramirez       0.229780
Q2_Sofia Ramirez       0.139476
Q3_Sofia Ramirez       0.147974
Q1_James Lee           0.145381
Q2_James Lee           0.170111
Q3_James Lee           0.103228
Q1_Lucas Hernandez     0.131305
Q2_Lucas Hernandez     0.131043
Q3_Lucas Hernandez     0.211921
Q1_Mia Gonzalez        0.412351
Q2_Mia Gonzalez        0.274824
Q3_Mia Gonzalez        0.182234
Q1_Liam Smith          0.097711
Q2_Liam Smith          0.240165
Q3_Liam Smith          0.097078
Q1_Olivia Brown        0.177696
Q2_Olivia Brown        0.128041
Q3_Olivia Brown        0.181161
Q1_Noah Wilson         0.184707
Q2_Noah Wilson         0.142347
Q3_Noah Wilson         0.211968
Q1_Ava Martinez        0.220901
Q2_Ava Martinez        0.218506
Q3_Ava Martinez        0.161556
Q1_William Thomas      0.332508
Q2_Willi

In [49]:
# Initialize a list to hold the new data
new_data = []

# Iterate through each row in the DataFrame
for index, row in similarity_df.iterrows():
    # Convert the row to a numeric type, ignoring errors
    row_numeric = pd.to_numeric(row, errors='coerce')

    # Sort the values in descending order while preserving the column names
    sorted_row = row_numeric.sort_values(ascending=False)

    # Filter out the person's own values (set to -1)
    sorted_row_filtered = sorted_row[sorted_row != -1]

    # Get the three highest values and their corresponding column names
    top_three_columns = sorted_row_filtered.head(3).index.tolist()
    top_three_values = sorted_row_filtered.head(3).tolist()

    # Extract the names and question numbers
    top_three_names = []
    top_three_questions = []
    for col in top_three_columns:
        parts = col.split('_', 1)
        if len(parts) == 2:
            question, name = parts
            top_three_names.append(name)
            top_three_questions.append(question)
        else:
            top_three_names.append(col)
            top_three_questions.append('')

    # Combine the names and similarity values into tuples
    top_three_combined = [(name, f'{value:.3f}', question) for name, value, question in zip(top_three_names, top_three_values, top_three_questions)]

    # Flatten the list of tuples and append it to new_data
    new_data.append([index] + [item for sublist in top_three_combined for item in sublist])

# Create columns names for the new DataFrame
columns = ['Name']
for i in range(1, 4):
    columns.extend([f'Pers{i}', f'Sim{i}', f'Quest{i}'])

# Create a new DataFrame from the new data
new_df = pd.DataFrame(new_data, columns=columns)

# Display or save the new DataFrame
print(new_df)

new_df.to_csv('top_matches_with_values.csv', index=False)


                Name            Pers1   Sim1 Quest1             Pers2   Sim2  \
0       Emma Johnson  Lucas Hernandez  0.483     Q2             Walid  0.481   
1        Arjun Patel     Sophia Davis  0.427     Q1   Benjamin Garcia  0.386   
2      Sofia Ramirez  Isabella Garcia  0.552     Q2  Elijah Rodriguez  0.454   
3          James Lee    Charlotte Lee  0.324     Q3      Sophia Davis  0.304   
4    Lucas Hernandez     Emma Johnson  0.564     Q2      Mia Gonzalez  0.447   
5       Mia Gonzalez            Walid  0.465     Q2    William Thomas  0.446   
6         Liam Smith  Lucas Hernandez  0.488     Q2      Emma Johnson  0.444   
7       Olivia Brown     Emma Johnson  0.403     Q3              Lola  0.394   
8        Noah Wilson    Charlotte Lee  0.318     Q3     Charlotte Lee  0.279   
9       Ava Martinez             Lola  0.427     Q2   Lucas Hernandez  0.405   
10    William Thomas            Walid  0.331     Q2      Sophia Davis  0.306   
11   Isabella Garcia    Sofia Ramirez  0

In [54]:
d_Quest = {'Q1':'Question1', 'Q2':'Question2', 'Q3':'Question3', 'Q4':'Question4', 'Q5':'Question5'}
lines = []
for index, row in new_df.iterrows():
    print(row['Name'])
    lines.append(row['Name'])
    filtered_df = df[df['Name'] == row['Name']]
    if not filtered_df.empty:
        lines.append(filtered_df[d_Quest[row['Quest1']]].iloc[0])
        lines.append(row['Pers1'])

        filtered_df_pers1 = df[df['Name'] == row['Pers1']]
        if not filtered_df_pers1.empty:
            lines.append(filtered_df_pers1['ExpertiseTweet'].iloc[0])


        lines.append(filtered_df[d_Quest[row['Quest2']]].iloc[0])
        lines.append(row['Pers2'])

        filtered_df_pers1 = df[df['Name'] == row['Pers2']]
        if not filtered_df_pers1.empty:
            lines.append(filtered_df_pers1['ExpertiseTweet'].iloc[0])

        lines.append(filtered_df[d_Quest[row['Quest3']]].iloc[0])
        lines.append(row['Pers3'])

        filtered_df_pers1 = df[df['Name'] == row['Pers3']]
        if not filtered_df_pers1.empty:
            lines.append(filtered_df_pers1['ExpertiseTweet'].iloc[0])

    break

Emma Johnson


In [55]:
lines[3]

'Building vibrant communities with #CommunityEngagement, #BrandAdvocacy, and #UserGeneratedContent. Connecting people and brands!'

In [56]:
lines

['Emma Johnson',
 'What strategies have you found most effective for building and maintaining a strong brand identity?',
 'Lucas Hernandez',
 'Building vibrant communities with #CommunityEngagement, #BrandAdvocacy, and #UserGeneratedContent. Connecting people and brands!',
 'What strategies have you found most effective for building and maintaining a strong brand identity?',
 'Walid',
 'I design and drive communities, connecting people between them and with the best opportunities.',
 'What strategies have you found most effective for building and maintaining a strong brand identity?',
 'Lola',
 'International marketer, building communities in education']

In [51]:
from openai import OpenAI

client = OpenAI(
  api_key=""
)

In [57]:
prompt = f"""I need a concise summary for each of the following:
1. {lines[2]}: {lines[3]}
2. {lines[5]}: {lines[6]}
3. {lines[8]}: {lines[9]}

Based on these summaries, please write a message for {lines[0]} saying, "Hey {lines[0]}, given your questions for the community, here is who you should meet: [summary of person 1] for your question about [summary of question 1], [summary of person 2] for your question about [summary of question 2], and [summary of person 3] for your question about [summary of question 3]." """

print(prompt)


I need a concise summary for each of the following:
1. Lucas Hernandez: Building vibrant communities with #CommunityEngagement, #BrandAdvocacy, and #UserGeneratedContent. Connecting people and brands!
2. Walid: I design and drive communities, connecting people between them and with the best opportunities.
3. Lola: International marketer, building communities in education

Based on these summaries, please write a message for Emma Johnson saying, "Hey Emma Johnson, given your questions for the community, here is who you should meet: [summary of person 1] for your question about [summary of question 1], [summary of person 2] for your question about [summary of question 2], and [summary of person 3] for your question about [summary of question 3]." 


In [ ]:
completion = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=prompt,
  max_tokens=150
)

print(completion.choices[0].text)



Hey Emma Johnson! 
We've identified some great community members for you to engage with in order to get the answers to the questions you've asked about what strategies have been found to be most effective for building and maintaining a strong brand identity. 
Lucas Hernandez is a great person to connect with - he specializes in building vibrant communities through community engagement, brand advocacy and user-generated content. 
Walid is also a great contact for you and can help you answer your questions - he designs and drives communities, aiming to connect people with the best opportunities.
Finally, Lola is the perfect person to reach out to since she specializes in international marketing and building communities in education. 
We hope these tips help you find


In [58]:
webhook_url = 'https://hook.eu1.make.com/rbqu7udzf9nkqltbmvp174svrtox4nji'

# Votre phrase et email
phrase = completion.choices[0].text
email = f'{lines[0]}@gmail.com'

# Préparation des données à envoyer
data = {
    'phrase': phrase,
    'email': email
}

# Envoi de la requête POST
response = requests.post(webhook_url, json=data)

# Vérification de la réponse
if response.status_code == 200:
    print("Succès : L'intro a été envoyés.")
else:
    print(f"Échec : Erreur {response.status_code}")

NameError: ignored

In [ ]:
def stable_roommates(preferences):
    n = len(preferences)
    roommates = {i: -1 for i in range(n)}  # Initialize roommates as unassigned (-1)
    proposals = {i: 0 for i in range(n)}  # Initialize proposals to keep track of current proposals

    while -1 in roommates.values():
        proposer = next((i for i, roommate in roommates.items() if roommate == -1), None)
        if proposer is None:
            break

        preferred_idx = preferences[proposer][proposals[proposer]]
        proposals[proposer] += 1

        if roommates[preferred_idx] == -1:  # If the preferred person is unassigned
            roommates[proposer] = preferred_idx
            roommates[preferred_idx] = proposer
        else:
            current_roommate = roommates[preferred_idx]
            if preferences[preferred_idx].index(proposer) < preferences[preferred_idx].index(current_roommate):
                roommates[proposer] = preferred_idx
                roommates[preferred_idx] = proposer
                roommates[current_roommate] = -1

    return roommates

# Preferences matrix for the given problem (using indices instead of names)
preferences = [
    [4, 1, 2, 3, 0],
    [0, 4, 5, 1, 2],
    [2, 0, 3, 4, 3],
    [3, 2, 4, 0, 2],
    [1, 5, 0, 2, 3],
    [1, 5, 0, 2, 3],
    [5, 0, 4, 3, 2],
    [2, 3, 1, 4, 5],
    [5, 3, 0, 4, 3],
    [0, 3, 4, 1, 9],
    [2, 3, 4, 11, 12],
    [2, 3, 4, 11, 12],
    [13, 3, 0, 1, 15],
    [5, 3, 14, 15, 0],
    [1, 0, 15, 4, 3],
    [2, 3, 11, 12, 5],
    [2, 3, 4, 11, 12]
]

# Solve the Stable Roommates Problem
roommate_assignments = stable_roommates(preferences)

# Print the stable roommate assignments
for proposer, preferred in roommate_assignments.items():
    print(f"Person {proposer} is roommates with Person {preferred}")


ValueError: ignored